# Efficient Implementation of ROC AUC Score

## Introduction

This post is a continuation of the [ROC and AUC Interpretation](https://maitbayev.github.io/posts/roc-auc). 
Please make sure that you understand that post before reading this post. 

In this post, we will implement an efficient ROC AUC Score in Python with $O(n\log n)$ runtime complexity.

[Subscribe](https://maitbayev.substack.com/subscribe) to get a notification about future posts. 

In [180]:
import numpy as np

np.random.seed(0)
n = 100
target = np.random.randint(0, 2, n)
predicted = np.random.rand(n)

In [181]:
import sklearn
sklearn.metrics.roc_auc_score(target, predicted)

np.float64(0.4277597402597403)

### Trapezoid Area

![Area of trapezoid](images/trapezoid.jpg)

We want to find the area of the trapezoid defined by the $(x_0, y_0)$ and $(x_1, y_1)$ points as shown in the picture above. We can add the area of the the rectangle and the right triangle, which is:

$$
\begin{align}
\text{Area}&=(x_1-x_0) \times y0+\frac{1}{2}(x_1-x_0) \times (y_1-y_0)\\
&= \frac{1}{2}(x_1-x_0) \times (2y_0+y_1 - y_0)\\
&= \frac{1}{2}(x_1-x_0) \times (y_0 + y_1)\\
\end{align}
$$

Let's implement the formula in Python:

In [187]:
def trapezoid_area(p0, p1):
    return (p1[0] - p0[0]) * (p0[1] + p1[1]) / 2.0

In [188]:
def fast_roc_auc_score(target, predicted):
    n = target.shape[0]
    num_positive = np.sum(target == 1)
    num_negative = n - num_positive 
    
    order = np.argsort(predicted)[::-1]
    last = [0, 0]
    num_true_positive = 0
    num_false_positive = 0
    score = 0
    for index in range(n):
        if index == 0 or predicted[order[index]] != predicted[order[index - 1]]:
            true_positive_rate = num_true_positive / num_positive
            false_positive_rate = num_false_positive / num_negative
            cur = [false_positive_rate, true_positive_rate]
            
            score += trapezoid_area(last, cur)
            last = cur
        
        if target[order[index]] == 1:
            num_true_positive += 1
        else:
            num_false_positive += 1
    score += trapezoid_area(last, [1, 1])
    return score 

In [189]:
fast_roc_auc_score(target, predicted)

np.float64(0.4277597402597403)